# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [13]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import prod
import os
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\ivanp\OneDrive\Documentos\INSPER\2o. Semestre_2022\C.Dados\22-2a-cd-p1-grupo_rafaeldbo\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [5]:
filename = '../data/dados.xlsx'

In [6]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
test = pd.read_excel(filename, sheet_name = 'Teste')

In [7]:
quantidade_train = train.value_counts('Target', sort=False)
quantidade_test = test.value_counts('Target',  sort=False)
# display(quantidade_train.to_frame())
# display(quantidade_test.to_frame())

quant_0 = quantidade_test[0]+quantidade_train[0]
quant_1 = quantidade_test[1]+quantidade_train[1]
quant_2 = quantidade_test[2]+quantidade_train[2]
quant_3 = quantidade_test[3]+quantidade_train[3]
total = sum(quantidade_train) + sum(quantidade_test)
total_train = sum(quantidade_train)
total_test = sum(quantidade_test)

print(f'No total, foram classificadas {total} notícias')
print(f"0: {quant_0} ({round(quant_0/total, 5)*100})")
print(f"1: {quant_1} ({round(quant_1/total, 5)*100})")
print(f"2: {quant_2} ({round(quant_2/total, 5)*100})")
print(f"3: {quant_3} ({round(quant_3/total, 5)*100})")

print(f'No total, foram classificadas {total_train} notícias na planilha de treinamento')
print(f"0: {quantidade_train[0]} ({round(quantidade_train[0]/total_train, 5)*100})")
print(f"1: {quantidade_train[1]} ({round(quantidade_train[1]/total_train, 5)*100})")
print(f"2: {quantidade_train[2]} ({round(quantidade_train[2]/total_train, 5)*100})")
print(f"3: {quantidade_train[3]} ({round(quantidade_train[3]/total_train, 5)*100})")

print(f'No total, foram classificadas {total_train} notícias na planilha de teste')
print(f"0: {quantidade_test[0]} ({round(quantidade_test[0]/total_test, 5)*100})")
print(f"1: {quantidade_test[1]} ({round(quantidade_test[1]/total_test, 5)*100})")
print(f"2: {quantidade_test[2]} ({round(quantidade_test[2]/total_test, 5)*100})")
print(f"3: {quantidade_test[3]} ({round(quantidade_test[3]/total_test, 5)*100})")


No total, foram classificadas 1400 notícias
0: 206 (14.713999999999999)
1: 310 (22.142999999999997)
2: 206 (14.713999999999999)
3: 678 (48.429)
No total, foram classificadas 1000 notícias na planilha de treinamento
0: 102 (10.2)
1: 248 (24.8)
2: 160 (16.0)
3: 490 (49.0)
No total, foram classificadas 1000 notícias na planilha de teste
0: 104 (26.0)
1: 62 (15.5)
2: 46 (11.5)
3: 188 (47.0)


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
#Limpador
stopWords = stopwords.words('portuguese')
lista_elementos= ',.@!#$%¨&*()–_-“”:;+‘’={[}]?/\|…""><'
def cleanup (texto):
    texto= texto.strip().lower()
    for letra in lista_elementos:
        texto= texto.replace(letra, " ")
    texto= texto.strip().split()
    for palavra in texto:
        if palavra in stopWords:
            texto.remove(palavra)
    return texto
train["Título_Limpo"]= train["Titulo"].apply(cleanup)
train["Descrição_Limpo"]= train["Descrição"].apply(cleanup)
train["Título_Limpo"]
train["Descrição_Limpo"]

0      [estudo, faculdade, medicina, universidade, ho...
1      [crescente, popularidade, impulsionado, aument...
2      [variante, ba, 2, cepa, ômicron, coronavírus, ...
3      [número, novos, casos, mortes, covid, 19, amér...
4      [plataforma, streaming, cria, regras, contra, ...
                             ...                        
995    [pudesse, decidir, que, preferiria, ir, a, lua...
996    [primeiro, africano, liderar, organização, mun...
997    [sabe, preguiça, dá, guardar, compras, você, c...
998    [juíza, responsável, decidir, bilionário, elon...
999    [vacinação, contra, covid, 19, crianças, 5, 11...
Name: Descrição_Limpo, Length: 1000, dtype: object

In [9]:
import pandas as pd
allWords = []
for wordsList in train['Descrição']:
    allWords += list(wordsList.split())
allWords = pd.Series(allWords)
portuguese = list(set(allWords))

In [24]:
#Fazendo o filtro de cada Target
filter_neutral= train["Target"] == 0
filter_very_bad= train["Target"] == 1
filter_bad= train["Target"] == 2
filter_good= train["Target"] == 3

#Aplicando o filtro na base de treinamento
train_neutral= train.loc[filter_neutral,:]
train_bad= train.loc[filter_bad,:]
train_very_bad= train.loc[filter_very_bad,:]
train_good= train.loc[filter_good,:]

#Criando a lista de palavras de cada target
list_neutral_words= train_neutral["Descrição_Limpo"].sum(axis= 0)
list_bad_words= train_bad["Descrição_Limpo"].sum(axis= 0)
list_very_bad_words= train_very_bad["Descrição_Limpo"].sum(axis= 0)
list_good_words= train_good["Descrição_Limpo"].sum(axis= 0)

#Criando um series para cada target
series_neutral= pd.Series(list_neutral_words)
series_bad= pd.Series(list_bad_words)
series_very_bad= pd.Series(list_very_bad_words)
series_good= pd.Series(list_good_words)

#Fazendo a frequencia absoluta
table_neutral= series_neutral.value_counts()
table_bad= series_bad.value_counts()
table_very_bad= series_very_bad.value_counts()
table_good= series_good.value_counts()

#Fazendo a probabilidade de cada target
prob_neutral= (len(list_neutral_words)/len(allWords))
prob_bad= (len(list_bad_words)/len(allWords))
prob_very_bad= (len(list_very_bad_words)/len(allWords))
prob_good= (len(list_good_words)/len(allWords))

# Suavizador de Laplace
def laplace_smoothing(wordsList, amount):
    laplace_amount = amount + 1
    density = len(wordsList) + len(portuguese)
    laplace = laplace_amount/density
    return laplace

#sendo wordsList = series de todas as palavras de um target
#sendo amount = quantidade de vezes que determinada palavra aparece lista do target (frequência absoluta)

#Função probabilidade do target 
def probability_target(table_target, series_target, news):
    
    list_target= []
    for word in news:
        if word in table_target.keys():
            amount= table_target[word]
        else:
            amount= 0
        list_target.append(laplace_smoothing(series_target, amount))
        
    probability_target= np.prod(list_target)
    return probability_target

#Função do Naive Bayes
def naive_bayes(news):
    prob_0= probability_target(table_neutral, series_neutral, news) * prob_neutral
    prob_1= probability_target(table_very_bad, series_very_bad, news) * prob_very_bad
    prob_2= probability_target(table_bad, series_bad, news) * prob_bad
    prob_3= probability_target(table_good, series_good, news) * prob_good
    list_probs= [prob_0,prob_1,prob_2,prob_3]
    target= list_probs.index(max(list_probs))
    return target
    
test["Descrição_Limpo"]= test["Descrição"].apply(cleanup)
test["Classificacao"]= test["Descrição_Limpo"].apply(naive_bayes)

pd.crosstab(test["Classificacao"],test["Target"],normalize=True).round(3) * 100


Target,0,1,2,3
Classificacao,,,,
0,3.5,0.0,0.2,0.5
1,3.0,10.2,4.0,9.0
2,2.0,1.0,1.5,0.8
3,17.5,4.2,5.8,36.8


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**